In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-apr-2022/train.csv")
label_df = pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")
test_df = pd.read_csv("../input/tabular-playground-series-apr-2022/test.csv")

# Feature Engineering

In [ ]:
from sklearn.preprocessing import StandardScaler

def get_sensor_cols(df):
    return [col for col in df if col.startswith('sensor')]

def preprocessing_df(df):
    sensor_cols = get_sensor_cols(df)
    feature_df = df.copy()
    n_lags = 2
    for sensor in sensor_cols:
        for lag in range(1, n_lags):
            feature_df[sensor + f'_lag{lag}'] = df.groupby('sequence')[sensor].shift(lag)
            feature_df[sensor + f'_diff{lag}'] = df.groupby('sequence')[sensor].diff(lag)
            feature_df = feature_df.fillna(0)
        
        for window in [3, 6, 12]:
            df[sensor + f'_roll_{window}_mean'] = df.groupby('sequence')[sensor]\
            .rolling(window=window, min_periods=1).mean().reset_index(level=0,drop=True)
            
            df[sensor + f'_roll_{window}_std'] = df.groupby('sequence')[sensor]\
            .rolling(window=window, min_periods=1).std().reset_index(level=0,drop=True)
            
            df[sensor + f'_roll_{window}_sum'] = df.groupby('sequence')[sensor]\
            .rolling(window=window, min_periods=1).sum().reset_index(level=0,drop=True)
        
    return feature_df

In [ ]:
sequences = train_df['sequence']
labels = label_df['state']

sc = StandardScaler()

features_train = preprocessing_df(train_df)
feature_cols = get_sensor_cols(features_train)
features_train[feature_cols] = sc.fit_transform(features_train[feature_cols])
features_train = features_train.drop(['sequence', 'subject', 'step'], axis=1).values
features_train = features_train.reshape(-1, 60, features_train.shape[-1])

features_test = preprocessing_df(test_df)
feature_cols = get_sensor_cols(features_test)
features_test[feature_cols] = sc.transform(features_test[feature_cols])
features_test = features_test.drop(['sequence', 'subject', 'step'], axis=1).values
features_test = features_test.reshape(-1, 60, features_test.shape[-1])

# 2) Bidirectional LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Bidirectional, BatchNormalization,
                                     Dense, Dropout, LSTM, GRU, Concatenate, Multiply,
                                     GlobalMaxPooling1D)
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.models import Model

In [ ]:
def create_lstm():
    x_input = Input(shape=(features_train.shape[-2:]))
    l1 = Bidirectional(LSTM(units=512, return_sequences=True))(x_input)
    l2 = Bidirectional(LSTM(units=256, return_sequences=True))(l1)
    g1 = Bidirectional(GRU(units=256, return_sequences=True))(l1)
    c1 = Concatenate(axis=2)([l2, g1])
    l3 = Bidirectional(LSTM(units=256, return_sequences=True))(c1)

    p1 = GlobalMaxPooling1D()(l3)

    d1 = Dense(units=128, activation='selu')(p1)
    
    x_output = Dense(units=1, activation='sigmoid')(d1)

    model = Model(inputs=x_input,
                  outputs=x_output,
                  name='lstm')
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics='AUC')
    return model

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

BATCH_SIZE = 512

k_fold = GroupKFold(n_splits=5)
valid_scores, predictions = [], []

for fold, (train_idx, valid_idx) in enumerate(k_fold.split(features_train,labels, sequences.unique())):
    
    X_train, X_valid = features_train[train_idx], features_train[valid_idx]
    y_train, y_valid = labels.iloc[train_idx].values, labels.iloc[valid_idx].values
    
    model = create_lstm()
    
    lr = ReduceLROnPlateau(monitor='val_auc', factor=0.05,
                          patience=5)
    
    es = EarlyStopping(monitor='val_auc', patience=10,
                       mode='max', restore_best_weights=True)
    
    model.fit(X_train, y_train,
              validation_data=(X_valid, y_valid),
              epochs=20,
              batch_size=BATCH_SIZE,
              callbacks=[lr, es],
              verbose=0)
    
    y_pred = model.predict(X_valid, batch_size=BATCH_SIZE).squeeze()
    
    roc_auc = roc_auc_score(y_valid, y_pred)
    valid_scores.append(roc_auc)
    predictions.append(model.predict(features_test, batch_size=BATCH_SIZE).squeeze())
    
    print(f'Fold-{fold + 1} ROC AUC: {roc_auc}')


In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')
submission['state'] = sum(predictions) / k_fold.n_splits
submission.to_csv('lstm.csv', index=False)
submission.head()

# 3) Keras Tuner

In [ ]:
def build_lstm(hp):
    x_input = Input(shape=(features_train.shape[-2:]))
    l1 = Bidirectional(LSTM(units=512,
                            return_sequences=True))(x_input)
    l2 = Bidirectional(LSTM(units=256,
                                return_sequences=True))(l1)
    
    for i in range(hp.Int('n_layers_lstm', min_value=2, max_value=5)):
        l2 = Bidirectional(LSTM(units=128,
                                return_sequences=True))(l2)
        
    g1 = Bidirectional(GRU(units=256,
                           return_sequences=True))(l1)
    
    for i in range(hp.Int('n_layers_gru', min_value=2, max_value=5)):
        g1 = Bidirectional(GRU(units=128,
                                return_sequences=True))(g1)
        
    c1 = Concatenate(axis=2)([l2, g1])

    l3 = Bidirectional(LSTM(units=256, return_sequences=True))(c1)

    p1 = GlobalMaxPooling1D()(l3)

    d1 = Dense(units=512,
               activation='selu')(p1)
    
    x_output = Dense(units=1, activation='sigmoid')(d1)

    model = Model(inputs=x_input,
                  outputs=x_output,
                  name='lstm')
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics='AUC')
    return model

In [ ]:
import keras_tuner as kt
tuner = kt.RandomSearch(
    hypermodel=build_lstm,
    objective=kt.Objective("val_auc", direction="max"),
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="kt",
    project_name='b_lstm'
)

In [ ]:
tuner.search_space_summary()

In [ ]:
import keras_tuner as kt

BATCH_SIZE = 512

k_fold = GroupKFold(n_splits=3)
valid_scores, predictions = [], []

for fold, (train_idx, valid_idx) in enumerate(k_fold.split(features_train,labels, sequences.unique())):
    
    X_train, X_valid = features_train[train_idx], features_train[valid_idx]
    y_train, y_valid = labels.iloc[train_idx].values, labels.iloc[valid_idx].values
    
    tuner = kt.RandomSearch(
        hypermodel=build_lstm,
        objective=kt.Objective("val_auc", direction="max"),
        max_trials=3,
        executions_per_trial=2,
        overwrite=True,
        directory="kt",
        project_name='b_lstm'
    )
    
    lr = ReduceLROnPlateau(monitor='val_auc', factor=0.05,
                          patience=5)
    
    es = EarlyStopping(monitor='val_auc', patience=5,
                       mode='max', restore_best_weights=True)
    
    tuner.search(X_train, y_train,
                 validation_data=(X_valid, y_valid),
                 epochs=1,
                 batch_size=BATCH_SIZE,
                 callbacks=[lr, es],
                 verbose=0)
    
    model = tuner.get_best_models(num_models=2)[0]
    y_pred = model.predict(X_valid, batch_size=BATCH_SIZE).squeeze()
    
    roc_auc = roc_auc_score(y_valid, y_pred)
    valid_scores.append(roc_auc)
    predictions.append(model.predict(features_test, batch_size=BATCH_SIZE).squeeze())
    
    print(f'Fold-{fold + 1} ROC AUC: {roc_auc}')


In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')
submission['state'] = sum(predictions) / k_fold.n_splits
submission.to_csv('lstm_tuned.csv', index=False)
submission.head()